<a href="https://colab.research.google.com/github/hwankang/Defense/blob/main/0704lstm_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#테디노트 (teddylee777.github.io/)
import pandas as pd
data_path='C:\Users\hwank\Downloads'
df_price = pd.read_csv(os.path.join(data_path, '005930.KS.csv'), encoding='utf8')

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd

data=pd.read_csv('005930.KS.csv', date_parser=True)
data.describe()

In [ ]:
data.head()

In [ ]:
#data['Date']=pd.to_datetime(data['Date'], format='%Y%m%d')
data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d %H:%M:%S')
data['year'] =data['Date'].dt.year
data['month'] =data['Date'].dt.month
data['day'] =data['Date'].dt.day
data.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(16, 9))

sns.lineplot(y=data['Close'], x=data['Date'])
plt.xlabel('time')
plt.ylabel('price')

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scale_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
data_scaled = scaler.fit_transform(data[scale_cols])

data_scaled = pd.DataFrame(data_scaled)
data_scaled.columns = scale_cols

print(data_scaled)

In [ ]:
TEST_SIZE=40
train = data_scaled[:-TEST_SIZE]
test = data_scaled[-TEST_SIZE:]
train.shape,test.shape

In [ ]:
import numpy as np
def make_dataset(data, label, window_size=20):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)

In [ ]:
feature_cols = ['Open', 'High', 'Low', 'Volume']
label_cols = ['Close']

train_feature = train[feature_cols]
train_label = train[label_cols]

# train dataset
train_feature, train_label = make_dataset(train_feature, train_label, 20)
train_feature.shape,train_label.shape

In [ ]:
test_feature = test[feature_cols]
test_label = test[label_cols]

# train, validation set 생성
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2)

x_train.shape, x_valid.shape
# ((6086, 20, 4), (1522, 20, 4))

# test dataset (실제 예측 해볼 데이터)
test_feature, test_label = make_dataset(test_feature, test_label, 20)
test_feature.shape, test_label.shape
# ((180, 20, 4), (180, 1))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM

model = Sequential()
model.add(LSTM(16, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               return_sequences=False)
          )
model.add(Dense(1))

In [ ]:
!pwd

In [ ]:
import os
model_path="/content/drive/My Drive"
model.compile(loss='mean_squared_error', optimizer='adam')
early_stop = EarlyStopping(monitor='val_loss', patience=5)
filename = os.path.join(model_path, 'tmp_checkpoint.h5')
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

history = model.fit(x_train, y_train, 
                    epochs=200, 
                    batch_size=16,
                    validation_data=(x_valid, y_valid), 
                    callbacks=[early_stop, checkpoint])

In [ ]:
model.load_weights(filename)

# 예측
pred = model.predict(test_feature)

In [ ]:
plt.figure(figsize=(12, 9))
plt.plot(test_label, label='actual')
plt.plot(pred, label='prediction')
plt.legend()
plt.show()